In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores.chroma import Chroma
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm_model = ChatOpenAI()
cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/1984.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

chain = RetrievalQA.from_chain_type(
    llm = llm_model,
    chain_type = "stuff",
    # 벡터 저장소를 retriever로 사용
    retriever = vectorstore.as_retriever(),
)

chain.run("Describe Victory Mansions")

'Victory Mansions is a building where Winston Smith resides. It has glass doors, a hallway that smells of boiled cabbage and old rag mats, and a colored poster of a large face with the caption "BIG BROTHER IS WATCHING YOU." The building has seven flights of stairs, and the elevator is rarely working. Inside Winston\'s flat, there is a telescreen that cannot be completely shut off. The building\'s atmosphere is stark and controlled, reflecting the oppressive nature of the society in George Orwell\'s novel "1984."'